# 패키지 불러오기

In [1]:
!openai --version

openai 1.35.7


In [2]:
import os
import time
import yaml
import re
import json

from openai import OpenAI
import pandas as pd
import ipywidgets as widgets # interactive display
from tqdm import tqdm # progress bar
from dotenv import load_dotenv

# ENV, Config 파일 읽기

In [3]:
# .env 파일에서 환경 변수 로드(API 키)
load_dotenv()

True

In [4]:
# YAML 파일 열기
yaml_path = 'config.yaml' # todo: config 파일과 합치기

with open(yaml_path, 'r') as f:
    config = yaml.safe_load(f)

# 인스턴스 생성

In [5]:
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

# 함수 정의

In [11]:
def get_response(messages):
    """
    model: 모델 종류
    messages: 사용자의 입력과 모델의 출력을 교환하는 메시지 목록
    max_tokens: 생성될 응답의 최대 길이
    temperature: 생성될 응답의 다양성(0.0 ~ 1.0) 0.0은 가장 확실한 답변을, 1.0은 가장 다양한 답변을 생성
    stream: 응답을 한 번에 반환할지 여부. False로 설정하면 한 번에 반환
    """
    response = client.chat.completions.create(
        model = "gpt-4o",
        messages = messages,
        # max_tokens = 150,
        temperature = 0.0,
        stream = False)
    return response.choices[0].message.content

# 파일 읽기

# test 영역

In [8]:
pwd()

'/Users/heungrokoh/python_workspace/work/voucher_paprika'

In [10]:
train = pd.read_csv('data/trainset_fine_tuning.csv')
train.head()

,tid(S사),raw_text,theme,issue,sentiment,sentiment(all)
0,FB-7,"자취하는데 뭔 티비야 .. 하면서 노트북이나 패드로 유튜브, ott 시청했었는데 확...","화면 크기, 이동 편리성, 삶의 질","큰 화면으로 OTT 시청, 이동 편리, 삶의 질 향상","화면 크기: 긍정, 이동 편리성: 긍정, 삶의 질: 긍정",긍정
1,FB-8,핸드폰으로 ott를 보다보니 항상 아쉬웠는데 고민고민하다질렀어요~^^ 주문후 다음날...,"배송, 디자인, 품질","빠른 배송, OTT 시청, 디자인 만족, 품질 만족","이동: 긍정, 사운드: 긍정, 화질: 긍정",긍정
2,FB-15,"너무 좋아요무겁지만 바퀴가 있어 이동이 용이 하고사운드,화질등 나무랄게 없어요","이동, 사운드, 화질","무겁지만 이동 용이, 우수한 사운드, 우수한 화질","이동: 긍정, 사운드: 긍정, 화질: 긍정",긍정
3,FB-16,밥 먹을때 자기전에 이동하면서 보고있는데 너무 편하고 좋아요,사용성,"이동 편리, 식사시간 시청, 자기전 시청",사용성: 긍정,긍정
4,FB-29,ott 유튜브만 보는분들에게 적극추천 리모컨으로도 터치로도 작동 잘됩니다~,"OTT/유튜브, 리모컨, 터치","OTT/유튜브 시청, 리모컨, 터치 작동","OTT/유튜브: 긍정, 리모컨: 긍정, 터치: 긍정",긍정


In [16]:
system_messages = """넌 글을 요약하는 전문가야. 내용을 입력하면 아래 지시사항에 따라 요약해줘.
지시사항:
1. 원문을 입력하면 주요 테마를 추출해줘.
2. 테마와 관련된 구체적인 이슈나 관심사를 나열해줘.
3. 테마별 감성 분석 결과를 나열해줘. 각 테마에 대해 매우 긍정, 긍정, 중립, 부정, 매우 부정 중 하나로 표현.
4. 원문 전체에 대한 감성 분석 결과를 매우 긍정, 긍정, 중립, 부정, 매우 부정 중 하나로 표현.
5. 결과를 포맷에 맞춰 출력해.

결과 포맷: 
[원문]
원문 텍스트

[theme]
원문에서 추출할 주요 테마를 나열

[issue]
테마와 관련된 구체적인 이슈나 관심사를 나열

[sentiment]
테마별 감성 분석 결과를 나열. 각 테마에 대해 매우 긍정, 긍정, 중립, 부정, 매우 부정 중 하나로 표현

[sentiment(all)]
원문 전체에 대한 감성 분석 결과를 매우 긍정, 긍정, 중립, 부정, 매우 부정 중 하나로 표현"""

In [20]:
# data폴더의 jsonl 파일 읽기
jsonl_path = 'data/fine_tuning_data.jsonl'
data = []
with open(jsonl_path, 'r', encoding='utf-8') as file:
    for line in file:
        data.append(json.loads(line))

# 데이터 출력 (예: 첫 번째 항목)
print(data[0])

# 또는 pandas DataFrame으로 변환
df = pd.DataFrame(data)

# DataFrame을 출력하여 쉽게 확인
print(df.head())


{'system_message': '넌 글을 요약하는 전문가야. 내용을 입력하면 아래 지시사항에 따라 요약해줘.\n지시사항:\n1. 원문을 입력하면 주요 테마를 추출해줘.\n2. 테마와 관련된 구체적인 이슈나 관심사를 나열해줘.\n3. 테마별 감성 분석 결과를 나열해줘. 각 테마에 대해 매우 긍정, 긍정, 중립, 부정, 매우 부정 중 하나로 표현.\n4. 원문 전체에 대한 감성 분석 결과를 매우 긍정, 긍정, 중립, 부정, 매우 부정 중 하나로 표현.\n5. 결과를 포맷에 맞춰 출력해.\n\n결과 포맷: \n[원문]\n원문 텍스트\n\n[theme]\n원문에서 추출할 주요 테마를 나열\n\n[issue]\n테마와 관련된 구체적인 이슈나 관심사를 나열\n\n[sentiment]\n테마별 감성 분석 결과를 나열. 각 테마에 대해 매우 긍정, 긍정, 중립, 부정, 매우 부정 중 하나로 표현\n\n[sentiment(all)]\n원문 전체에 대한 감성 분석 결과를 매우 긍정, 긍정, 중립, 부정, 매우 부정 중 하나로 표현', 'user_message': '[원문]\n자취하는데 뭔 티비야 .. 하면서 노트북이나 패드로 유튜브, ott 시청했었는데 확실히 더 큰 화면으로 보니까 너무 조아요설거지하거나 가구 배치 바꿀 때도 쉽게 움직여서 이동시킬 수 있으니까 정말 좋습니다..삶의 질이 향상됐어요 ㅎㅎ\n[/원문]', 'assistant_message': '[원문]\n자취하는데 뭔 티비야 .. 하면서 노트북이나 패드로 유튜브, ott 시청했었는데 확실히 더 큰 화면으로 보니까 너무 조아요설거지하거나 가구 배치 바꿀 때도 쉽게 움직여서 이동시킬 수 있으니까 정말 좋습니다..삶의 질이 향상됐어요 ㅎㅎ\n[/원문]\n[theme]\n화면 크기, 이동 편리성, 삶의 질\n[/theme]\n[issue]\n큰 화면으로 OTT 시청, 이동 편리, 삶의 질 향상\n[/issue]\n[sentiment]\n화면 크기: 긍정, 이동 편리성: 긍정, 삶의 질: 긍정\n[/sentiment]

In [ ]:
results_list = []
# 테스트 데이터: test
total_iterations = len(test)

with tqdm(total=total_iterations, desc="Processing") as pbar:
    for i in test:
        messages = [
            {"role": "system", "content": config['gptapi']['system_content']['role_summarize']},
            {"role": "user", "content": i},
        ]
        # 요약 결과를 받아오기
        response_sum = get_response(messages)
        messages.append({"role": "assistant", "content": response_sum})
        # 감정 분석 결과를 받아오기
        input_senti = config['gptapi']['assistant_content']['senti_categ']
        messages.append({"role": "user", "content": input_senti})
        response_senti = get_response(messages)

        results_list.append({'original_message': i, 'summary': response_sum, 'sentiment': response_senti})
        pbar.update(1)
        time.sleep(1)

results = pd.DataFrame(results_list)

Processing: 100%|██████████| 10/10 [00:33<00:00,  3.36s/it]
